In [1]:
import torch
from   torch import nn
from   math  import log2
from   torch import optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
from   torch.utils.data import DataLoader

import matplotlib.pyplot      as plt
import matplotlib.animation   as animation
import numpy as np

from model import Generator
from model import Critic
from tqdm  import tqdm

In [2]:
DATA_DIR = "../data/flikr_hq/"
START_IMG_SIZE = 4
GEN_CHECKPOINT = "gen.pth"
CRIT_CHECKPOINT = "crit.pth"
DEVICE = "cuda"
SAVE_MODEL = True
LOAD_MODEL = False
LEARNING_RATE = 1e-3
BATCH_SIZE = 64
IMG_SIZE = 1024
IMG_CHANNELS = 3
Z_DIM = 512
IN_CHANNELS = 512
LAMBDA_GP = 10
NUM_STEPS = int(log2(IMG_SIZE / 4)) + 1
PROG_EPOCHS = [30] * 9
FIXED_NOISE = torch.randn(8, Z_DIM, 1, 1).to(DEVICE)
NUM_WORKERS = 32
DISPLAY_STEP = 500

torch.backends.cudnn.benchmarks = True

In [3]:
def gradient_penalty(critic, real, fake, alpha, train_step, device):
    batch_size, img_channels, h, w = real.shape
    beta = torch.rand((batch_size, 1, 1, 1)).repeat(1, img_channels, h, w).to(device)
    mixed_imgs = real * beta + fake.detach() * (1 - beta)
    mixed_imgs.requires_grad_(True)
    mixed_scores = critic(mixed_imgs, alpha, train_step)

    gradient = torch.autograd.grad(
        inputs=mixed_imgs,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True
    )[0]

    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    penalty = torch.mean((gradient_norm - 1) ** 2)
    
    return penalty


In [4]:
def save_checkpoint(model, optim, filename="progan_ckpnt.pth"):
    print("### Saving checkpoint ###")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optim.state_dict()
    }
    torch.save(checkpoint, filename)


def load_checkpoint(filename, model, optim, lr):
    print("### Loading checkpoint ###")
    checkpoint = torch.load(filename, map_location="cuda")
    model.load_state_dict(checkpoint["state_dict"])
    optim.load_state_dict(checkpoint["optimizer"])

    for group in optim.param_groups:
        group["lr"] = lr

In [5]:
def get_loader(img_size):
    transform = transforms.Compose([
        transforms.Resize(img_size, img_size),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(IMG_CHANNELS)],
            [0.5 for _ in range(IMG_CHANNELS)]
        )
    ])

    dataset = datasets.ImageFolder(root=DATA_DIR, transform=transform)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

    return loader, dataset

In [6]:
gen = Generator(Z_DIM, IN_CHANNELS, IMG_CHANNELS).to(DEVICE)
crit = Critic(IN_CHANNELS, IMG_CHANNELS).to(DEVICE)

gen_optim = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.99))
crit_optim = optim.Adam(crit.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.99))

gen_scalar = torch.cuda.amp.GradScaler()
crit_scalar = torch.cuda.amp.GradScaler()

if LOAD_MODEL:
    load_checkpoint(GEN_CHECKPOINT, gen, gen_optim, LEARNING_RATE)
    load_checkpoint(CRIT_CHECKPOINT, crit, crit_optim, LEARNING_RATE)

gen.train()
crit.train()

step = int(log2(START_IMG_SIZE / 4))

img_list = []

for epochs in PROG_EPOCHS[step:]:
    alpha = 0
    loader, dataset = get_loader(4 * 2 ** step)
    print(f"Image size: {4 * 2 ** step}")

    for epoch in range(epochs):
        print(f"Epoch [{epoch + 1}/{epochs}]")

        # alpha = train(gen, crit, loader, dataset, step, alpha, gen_optim, crit_optim, gen_scalar, crit_scalar, epoch)
        loop = tqdm(loader, leave=True)
        for batch_idx, (real, _) in enumerate(loop):
            real = real.to(DEVICE)
            cur_batch_size = real.shape[0]

            # Train critic
            noise = torch.randn(cur_batch_size, Z_DIM, 1, 1).to(DEVICE)

            with torch.cuda.amp.autocast():
                fake = gen(noise, alpha, step)
                real_pred = crit(real, alpha, step)
                fake_pred = crit(fake.detach(), alpha, step)
                gp = gradient_penalty(crit, real, fake, alpha, step, DEVICE)
                crit_loss = -(torch.mean(real_pred) - torch.mean(fake_pred)) + LAMBDA_GP * gp + (0.001 * torch.mean(real_pred ** 2))

            crit_optim.zero_grad()
            crit_scalar.scale(crit_loss).backward()
            crit_scalar.step(crit_optim)
            crit_scalar.update()

            # Train generator
            with torch.cuda.amp.autocast():
                gen_fake = crit(fake, alpha, step)
                gen_loss = -torch.mean(gen_fake)

            gen_optim.zero_grad()
            gen_scalar.scale(gen_loss).backward()
            gen_scalar.step(gen_optim)
            gen_scalar.update()

            alpha += cur_batch_size / (PROG_EPOCHS[step] * 0.5) + len(dataset)
            alpha = min(alpha, 1)

            if batch_idx % DISPLAY_STEP == 0 and batch_idx > 0:
                with torch.no_grad():
                    fake = gen(FIXED_NOISE, alpha, step).detach().cpu()
                img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        if SAVE_MODEL:
            save_checkpoint(gen, gen_optim, filename=GEN_CHECKPOINT)
            save_checkpoint(crit, crit_optim, filename=CRIT_CHECKPOINT)

    step += 1

Image size: 4
Epoch [1/30]


 41%|████      | 448/1094 [00:46<01:07,  9.60it/s]


KeyboardInterrupt: 